### Apriori算法实现

In [1]:
import numpy as np
import pandas as pd

In [2]:
#辅助函数
def createC1(dataset):
    '''
    对数据集生成1-项集
    Args:
        dataset:输入数据,list
    '''
    C1=[]
    for data in dataset:
        for item in data:
            if [item] not in C1:
                C1.append([item])
    C1.sort()
    #frozenset冻结的集合，即不可变的集合，因为之后要作为字典的键
    return list(map(frozenset,C1))

def scanD(D,Ck,minSupport):
    '''
    计算每个项集的支持度
    Args:
        D(List):输入的数据库
        Ck:输入的项集
        minSupport:最小支持度阈值
    Returns:
        retList(List):满足条件的频繁项集
        supportData(Dict):每个项集和它对应的支持度
    '''
    ssCnt={}
    #统计每个项集在数据集中出现的次数
    for c in Ck:
        for data in D:
            if c.issubset(data):
                if c not in ssCnt:
                    ssCnt[c]=1
                else:
                    ssCnt[c]+=1
    #数据总量
    num=float(len(D)) 
    retList=[]
    supportData={}
    #计算每个项集的支持度
    for key in ssCnt:
        support=ssCnt[key]/num
        if support>=minSupport:
            retList.insert(0,key)
        supportData[key]=support
    #返回满足条件的项集和所有项集的支持度字典
    return retList,supportData

In [3]:
#测试
data=[[1,3,4],[2,3,5],[1,2,3,5],[2,5]]
C1=createC1(data)
L1,supportData0=scanD(data,C1,0.5)
print(C1)
print(L1)
print(supportData0)

[frozenset({1}), frozenset({2}), frozenset({3}), frozenset({4}), frozenset({5})]
[frozenset({5}), frozenset({3}), frozenset({2}), frozenset({1})]
{frozenset({1}): 0.5, frozenset({2}): 0.75, frozenset({3}): 0.75, frozenset({4}): 0.25, frozenset({5}): 0.75}


#### 完整的Apriori算法

整个Apriori算法的伪代码如下:
* 当集合中项的个数大于0时
    * 构建一个k个项组成的候选项集的列表
    * 检查数据以确认每个项集都是频繁的
    * 保留频繁项集并构建k+1项组成的候选项集的列表

In [4]:
def aprioriGen(Lk,k):
    '''
    将项集两两合并
    Args:
        Lk(set):输入的项集
        k:项集元素个数k
    Returns:
        retList:合并后得到的k项集
    '''
    retList=[]
    lenLk=len(Lk)
    for i in range(lenLk):
        for j in range(i+1,lenLk):
            L1=list(Lk[i])
            L2=list(Lk[j])
            L1.sort()
            L2.sort()
            #只将除了最后一个元素，前面元素都相同的合并
            #比如{0,1},{0,2},{1,2}
            #若不这样则会得到三个{0,1,2}
            if L1[:-1]==L2[:-1]:
                retList.append(Lk[i]|Lk[j])

    return retList

def apriori(dataSet,minSupport=0.5):
    '''
    Apriori算法核心函数
    Args:
        dataset:输入的数据
        minSupport:最小支持度阈值
    Returns:
        L:包含L1,L2,..Lk的列表
        supportData:记录各项集支持度的字典
    '''
    C1=createC1(dataSet)
    D=list(map(set,dataSet))
    #L1为全为1-项集的集合
    L1,supportData=scanD(D,C1,minSupport)
    #L将包含L1,L2,..Lk
    L=[L1]
    k=2
    #当Lk为空时，循环退出
    while((len(L[-1]))>0):
        Ck=aprioriGen(L[-1],k)
        Lk,supK=scanD(D,Ck,minSupport)
        supportData.update(supK)
        L.append(Lk)
        k+=1
    return L,supportData

In [5]:
L,supportData=apriori(data,0.5)
print(L)
print(supportData)

[[frozenset({5}), frozenset({3}), frozenset({2}), frozenset({1})], [frozenset({1, 3}), frozenset({2, 3}), frozenset({2, 5}), frozenset({3, 5})], [frozenset({2, 3, 5})], []]
{frozenset({1}): 0.5, frozenset({2}): 0.75, frozenset({3}): 0.75, frozenset({4}): 0.25, frozenset({5}): 0.75, frozenset({3, 5}): 0.5, frozenset({2, 5}): 0.75, frozenset({1, 5}): 0.25, frozenset({2, 3}): 0.5, frozenset({1, 3}): 0.5, frozenset({1, 2}): 0.25, frozenset({2, 3, 5}): 0.5}


### 根据Apriori算法生成规则

假设规则{A,B,C}->{D}不满足最小可信度要求，那么所有已{A,B,C}子集为规则左部的也不会满足最小可信度要求，即{A,B}->{C,D},{A,C}->{B,D},{B,C}->{A,D},{A}->{B,C,D},{B}->{A,C,D},{C}->{A,B,D}.

以{A,B}->{C,D}为例说明:

因为{A,B,C}->{D}可信度小于阈值，即
$$
\frac{support(A,B,C,D)}{support(A,B,C)}<min\_support
$$
又因为
$$
support(A,B)>=support(A,B,C)
$$
所以
$$
\frac{support(A,B,C,D)}{support(A,B)}<\frac{support(A,B,C,D)}{support(A,B,C)}<min\_support
$$

利用该性质来减少计算的规则数目。

In [6]:
#参考:https://github.com/stonycat/ML-in-Action-Code-and-Note/blob/master/ch11/apriori.py
def generateRules(L,supportData,minConf=0.7):
    '''
    生成规则的核心函数
    Args:
        L:
        supportData:
        minConf:
    Returns:
        
    '''
    bigRuleList=[]
    for i in range(1, len(L)):  #只获取有两个或者更多集合的项目，从1,即第二个元素开始，L[0]是单个元素的
        # 两个及以上的才可能有关联一说，单个元素的项集不存在关联问题
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            #该函数遍历L中的每一个频繁项集并对每个频繁项集创建只包含单个元素集合的列表H1
            if (i > 1):
            #如果频繁项集元素数目超过2,那么会考虑对它做进一步的合并
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:#第一层时，后件数为1
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)# 调用函数2
    return bigRuleList  

#生成候选规则集合：计算规则的可信度以及找到满足最小可信度要求的规则
def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    #针对项集中只有两个元素时，计算可信度
    prunedH = []#返回一个满足最小可信度要求的规则列表
    for conseq in H:#后件，遍历 H中的所有项集并计算它们的可信度值
        conf = supportData[freqSet]/supportData[freqSet-conseq] #可信度计算，结合支持度数据
        if conf >= minConf:
            print (freqSet-conseq,'-->',conseq,'conf:',conf)
            #如果某条规则满足最小可信度值,那么将这些规则输出到屏幕显示
            brl.append((freqSet-conseq, conseq, conf))#添加到规则里，brl 是前面通过检查的 bigRuleList
            prunedH.append(conseq)#同样需要放入列表到后面检查
    return prunedH

#合并
def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    #参数:一个是频繁项集,另一个是可以出现在规则右部的元素列表 H
    m = len(H[0])
    if (len(freqSet) > (m + 1)): #频繁项集元素数目大于单个集合的元素数
        Hmp1 = aprioriGen(H, m+1)#存在不同顺序、元素相同的集合，合并具有相同部分的集合
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)#计算可信度
        if (len(Hmp1) > 1):    #满足最小可信度要求的规则列表多于1,则递归
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)

In [7]:
#测试
rules=generateRules(L,supportData,minConf=0.7)

frozenset({1}) --> frozenset({3}) conf: 1.0
frozenset({5}) --> frozenset({2}) conf: 1.0
frozenset({2}) --> frozenset({5}) conf: 1.0


In [8]:
rules=generateRules(L,supportData,minConf=0.5)

frozenset({3}) --> frozenset({1}) conf: 0.6666666666666666
frozenset({1}) --> frozenset({3}) conf: 1.0
frozenset({3}) --> frozenset({2}) conf: 0.6666666666666666
frozenset({2}) --> frozenset({3}) conf: 0.6666666666666666
frozenset({5}) --> frozenset({2}) conf: 1.0
frozenset({2}) --> frozenset({5}) conf: 1.0
frozenset({5}) --> frozenset({3}) conf: 0.6666666666666666
frozenset({3}) --> frozenset({5}) conf: 0.6666666666666666
frozenset({5}) --> frozenset({2, 3}) conf: 0.6666666666666666
frozenset({3}) --> frozenset({2, 5}) conf: 0.6666666666666666
frozenset({2}) --> frozenset({3, 5}) conf: 0.6666666666666666
